# CSV Analyzer Agent
Loads and summarizes a CSV using a tool.

In [ ]:
import pandas as pd
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor

@tool
def load_csv(path: str):
    df = pd.read_csv(path)
    return df.describe().to_string()

llm = ChatOpenAI(model='gpt-4o-mini')
agent = create_react_agent(llm, [load_csv], 'Use load_csv to inspect data.')
executor = AgentExecutor(agent=agent, tools=[load_csv])
executor.invoke({'input':'Load the CSV at sample.csv'})